In [8]:
import pandas as pd
from binance.client import Client
from Binance_keys import api_key, secret_key
from datetime import datetime, timedelta
import time
from binance.exceptions import *
from auxiliary_functions import *

client = Client(api_key, secret_key)

In [ ]:
# import pandas as pd
# from binance.client import Client
# from Binance_keys import api_key, secret_key
# from datetime import datetime, timedelta
# import time
# client = Client(api_key, secret_key, tld='us')

In [9]:
def truncate(number, decimals=0):
    
    if not isinstance(decimals, Int):
         raise TypeError("decimal places must be an integer.")
    elif decimals < e:
        raise ValueError("decimal places has to be e or nore.")
    elif decimals== 0:
         return math.trunc (number)
    factor = 10.0 ** decimals
    return math. trunc (number*factor) / factor
                  
    
def sma(data, window):
    return(data.rolling(window = window).mean())

def bollinger_band(data, sma, window, nstd):
    std = data.rolling (window = window).std()
    upper_band = sma + std * nstd
    lower_band = sma - std * nstd
    return upper_band, lower_band

In [10]:
symbols = ['BTC','ETH', 'LTC' ]
start_n_hours_ago = 48


def gather_data(symbols, start_n_hours_ago):

    merge = False
    for symbol in symbols:
        klines = client.get_historical_klines(symbol= f'{symbol}USDT',interval=client.KLINE_INTERVAL_1MINUTE,
                                              start_str=str(datetime.now()-timedelta(hours=start_n_hours_ago)))

        cols = ['OpenTime',
                f'{symbol}-USD_Open',
                f'{symbol}-USD_High',
                f'{symbol}-USD_Low',
                f'{symbol}-USD_Close',
                f'{symbol}-USD_volume',
                 'CloseTime',
                f'{symbol}-QuoteAssetvolume',
                f'{symbol}-NunberofTrades',
                f'{symbol}-TBBAV',
                f'{symbol}-TBQAV',
                 f'{symbol}-ignore']

        df = pd.DataFrame (klines, columns=cols)
        if merge==True:
            dfs = pd.merge(df, dfs, how='inner',on=['OpenTime','CloseTime'])
        else:
            dfs = df
            merge = True
    dfs['OpenTime'] = [datetime.fromtimestamp(ts / 1000) for ts in dfs['OpenTime']]
    dfs['CloseTime'] = [datetime.fromtimestamp(ts / 1000) for ts in dfs['CloseTime']]
    for col in dfs.columns:
        if not 'Time' in col:
            dfs[col] = dfs[col].astype(float)

    for symbol in symbols:
        dfs[f'{symbol}_sma'] = sma(dfs[f'{symbol}-USD_Close'],window=20)
        dfs[f'{symbol}_upper_band'], dfs[f'{symbol}_lower_band'] = bollinger_band (data=dfs[f'{symbol}-USD_Close'],
                                                                      sma=dfs[f'{symbol}_sma'],
                                                                      window=20, nstd=3)

    dfs.dropna(inplace=True)

    return dfs



def get_states(df, synbols):
    states={}
    
    for symbol in symbols:
        if df[f'{symbol}-USD_Close'].iloc[-1] < df[f'{symbol}_lower_band'].iloc[-1]:
              states[synbol] = 'below'
        elif df[f'{symbol}-USD_Close'].iloc[-1] > df[f'{symbol}_upper_band'].iloc[-1]:
              states[symbol] = 'above'
        else:
              states[symbol] = 'inside'
    return states

In [11]:
symbols = ['BTC','ETH','LTC']
start_n_hours_ago = 48
balance_unit = 'USDT'
first = True

BUY_AMOUNT_USDT = 100

while True:
    if (datetime.now().second % 10 == 0) or first:
        if (datetime.now().minute == 0 and datetime.now().second == 10) or first:
            # refresh data
            first = False
            df = gather_data(symbols,48)
            states = get_states(df,symbols)
            print('Current state of the market:')
            print(states)
            print('\n')

        try:
            if balance_unit == 'USDT': # looking to buy
                for symbol in symbols:
                    ask_price = float(client.get_orderbook_ticker(symbol = f'{symbol}USDT')['askPrice'])
                    lower_band = df[f'{symbol}_lower_band'].iloc[-1]
                    
                    if ask_price < lower_band and states[symbol] == 'inside': #buy signal
                        ######################
                        print(f'Buy order placed:')
                        precision = get_precision(price=ask_price,desired_amount_usdt=BUY_AMOUNT_USDT)
                        buy_order = client.order_limit_buy(symbol=f'{symbol}USDT',
                                                          quantity=truncate(BUY_AMOUNT_USDT / ask_price, precision),
                                                          price = ask_price)
                        print(buy_order)

                        start = datetime.now()
                        while True:
                            time.sleep(1)
                            buy_order = client.get_order(symbol=buy_order['symbol'], orderId=buy_order['orderId'])

                            seconds_since_buy = (datetime.now() - start).seconds

                            # resolve buy order
                            if float(buy_order['executedQty']) == 0 and seconds_since_buy > 60*60:
                                # no fill
                                client.cancel_order(symbol=buy_order['symbol'], orderId=buy_order['orderId'])
                                print('Order not filled after 1 hour, cancelled.')
                                print('\n')
                                break

                            if float(buy_order['executedQty']) != 0 and float(buy_order['executedQty']) != float(buy_order['origQty']) and seconds_since_buy > 60*60:
                                # partial fill
                                client.cancel_order(symbol=buy_order['symbol'], orderId=buy_order['orderId'])
                                balance_unit = symbol
                                print('Order partially filled after 1 hour, cancelled the rest and awaiting sell signal.')
                                print('\n')
                                break

                            if float(buy_order['executedQty']) ==  float(buy_order['origQty']):
                                # completely filled
                                balance_unit = symbol
                                print('Order filled:')
                                print(buy_order)
                                print('\n')
                                break
                        
                        ######################

            if balance_unit != 'USDT': # looking to sell
                bid_price = float(client.get_orderbook_ticker(symbol = f'{balance_unit}USDT')['bidPrice'])
                upper_band = df[f'{balance_unit}_upper_band'].iloc[-1]
                if bid_price > upper_band and states[balance_unit] == 'inside': #sell signal
                    ######################
                    # use same precision as buy order
                    client.order_market_sell(symbol=buy_order['symbol'],
                                            quantity=truncate(float(buy_order['executedQty']), precision))
                    
                    ######################
                    balance_unit = 'USDT'

            time.sleep(1)
            
        except BinanceAPIException as e:
            print(e.status_code)
            print(e.message)

Current state of the market:
{'BTC': 'inside', 'ETH': 'inside', 'LTC': 'inside'}


Current state of the market:
{'BTC': 'inside', 'ETH': 'inside', 'LTC': 'inside'}


Buy order placed:


NameError: name 'Int' is not defined